In [1]:
!pip install transformers accelerate torch --quiet
!pip install -q huggingface_hub
!pip install optimum gptqmodel
!pip install -U bitsandbytes accelerate
!pip install peft

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
import torch

  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None,
    num_labels=2
)

df = pd.read_parquet("/content/samples_3k_project_c_updated.parquet")

df.drop(['emails', 'base_emails', 'brand', 'base_brand'], axis=1)

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

train_df.to_json("train.jsonl", orient="records", lines=True)
test_df.to_json("test.jsonl", orient="records", lines=True)

train_data = load_dataset("json", data_files="train.jsonl", split="train")
test_data = load_dataset("json", data_files="test.jsonl", split="train")

def tokenize_data(data):
    prompt = (f"Record A: {data['names']} | {data['websites']} | {data['socials']} | {data['emails']} | {data['phones']} | {data['addresses']}\n"
        f"Record B: {data['base_names']} | {data['base_websites']} | {data['base_socials']} | {data['base_emails']} | {data['base_phones']} | {data['base_addresses']}\n"
    )
    tokenized = tokenizer(prompt, max_length=256, truncation=True)
    tokenized["labels"] = int(data["label"])
    tokenized["id"] = data["id"]
    return tokenized

train_data = train_data.map(tokenize_data)
test_data = test_data.map(tokenize_data)

def fix_data(data):
    data["labels"] = int(data["labels"])
    return data

train_data = train_data.map(fix_data)
test_data = test_data.map(fix_data)
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import numpy as np

counts = df["label"].value_counts()
print(counts)

print("\nPercentage:")
print(counts / counts.sum() * 100)


label
1.0    1809
0.0    1191
Name: count, dtype: int64

Percentage:
label
1.0    60.3
0.0    39.7
Name: count, dtype: float64


Meta's LLama

In [10]:
from huggingface_hub import login
login()

In [11]:
model_name = "meta-llama/Llama-3.2-1B"

import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None,
    num_labels=2
)

df = pd.read_parquet("/content/samples_3k_project_c_updated.parquet")

df.drop(['emails', 'base_emails', 'brand', 'base_brand'], axis=1)

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

train_df.to_json("train.jsonl", orient="records", lines=True)
test_df.to_json("test.jsonl", orient="records", lines=True)

train_data = load_dataset("json", data_files="train.jsonl", split="train")
test_data = load_dataset("json", data_files="test.jsonl", split="train")

def tokenize_data(data):
    prompt = (f"Record A: {data['names']} | {data['websites']} | {data['socials']} | {data['emails']} | {data['phones']} | {data['addresses']}\n"
        f"Record B: {data['base_names']} | {data['base_websites']} | {data['base_socials']} | {data['base_emails']} | {data['base_phones']} | {data['base_addresses']}\n"
    )
    tokenized = tokenizer(prompt, max_length=256, truncation=True)
    tokenized["labels"] = int(data["label"])
    tokenized["id"] = data["id"]
    return tokenized

train_data = train_data.map(tokenize_data)
test_data = test_data.map(tokenize_data)

def fix_data(data):
    data["labels"] = int(data["labels"])
    return data

train_data = train_data.map(fix_data)
test_data = test_data.map(fix_data)
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

model.to(device)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

...

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=

Google's Electra

In [2]:
model_name = "google/electra-small-discriminator"

import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": device},
    num_labels=2
)

df = pd.read_parquet("/content/samples_3k_project_c_updated.parquet")

df.drop(['emails', 'base_emails', 'brand', 'base_brand'], axis=1)

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

train_df.to_json("train.jsonl", orient="records", lines=True)
test_df.to_json("test.jsonl", orient="records", lines=True)

train_data = load_dataset("json", data_files="train.jsonl", split="train")
test_data = load_dataset("json", data_files="test.jsonl", split="train")

def tokenize_data(data):
    prompt = (f"Record A: {data['names']} | {data['websites']} | {data['socials']} | {data['emails']} | {data['phones']} | {data['addresses']}\n"
        f"Record B: {data['base_names']} | {data['base_websites']} | {data['base_socials']} | {data['base_emails']} | {data['base_phones']} | {data['base_addresses']}\n"
    )
    tokenized = tokenizer(prompt, max_length=256, truncation=True)
    tokenized["labels"] = int(data["label"])
    tokenized["id"] = data["id"]
    return tokenized

train_data = train_data.map(tokenize_data)
test_data = test_data.map(tokenize_data)

def fix_data(data):
    data["labels"] = int(data["labels"])
    return data

train_data = train_data.map(fix_data)
test_data = test_data.map(fix_data)
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

In [4]:
# tokenizer.pad_token = tokenizer.eos_token #for llama
# model.config.pad_token_id = tokenizer.pad_token_id #for llama
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "test-trainer",
    report_to="none",
    bf16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-2036784160.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.529900


TrainOutput(global_step=789, training_loss=0.47007706984216574, metrics={'train_runtime': 44.0557, 'train_samples_per_second': 143.001, 'train_steps_per_second': 17.909, 'total_flos': 92545915783488.0, 'train_loss': 0.47007706984216574, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("/content/drive/MyDrive/no-lora-electra")
tokenizer.save_pretrained("/content/drive/MyDrive/no-lora-electra")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/no-lora-electra/tokenizer_config.json',
 '/content/drive/MyDrive/no-lora-electra/special_tokens_map.json',
 '/content/drive/MyDrive/no-lora-electra/tokenizer.json')

In [5]:
import numpy as np
# testing data
test_data = test_data.map(tokenize_data)
results = trainer.predict(test_data)
# print(results.label_ids)
argmax_preds = torch.argmax(torch.tensor(results.predictions), dim=-1).numpy()
print(argmax_preds[:20])
# print(argmax_preds)
print(f"Accuracy: {np.mean(results.label_ids == argmax_preds)}")

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

[1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0]
Accuracy: 0.8355555555555556


In [6]:
tp_count = 0
fp_count = 0
tn_count = 0
fn_count = 0
for x in range(len(results.label_ids)):
  if (results.label_ids[x] == 1 and argmax_preds[x] == 1):
    tp_count += 1
  elif (results.label_ids[x] == 0 and argmax_preds[x] == 0):
    tn_count += 1
  elif (results.label_ids[x] == 0 and argmax_preds[x] == 1):
    fn_count += 1
  else:
    fp_count += 1

print("True Positive: ", tp_count)
print("True Negative: ", tn_count)
print("TP:FP - ", float(tp_count)/fp_count)
print("False Positive: ", fp_count)
print("False Negative: ", fn_count)
print("TN:FN - ", float(tn_count)/fn_count)
precision = tp_count/(tp_count + fp_count)
print("Precision: ", precision)
recall = tp_count/(tp_count + fn_count)
print("Recall: ", recall)
print("F1 Score: ", 2*float(precision)*recall/(precision+recall))

True Positive:  460
True Negative:  292
TP:FP -  4.8936170212765955
False Positive:  94
False Negative:  54
TN:FN -  5.407407407407407
Precision:  0.8303249097472925
Recall:  0.8949416342412452
F1 Score:  0.8614232209737828
